In [1]:
%load_ext Cython

In [9]:
%%cython

import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport exp
from libc.stdlib cimport rand, srand, RAND_MAX
from guthrie_modelc import *

cdef double[:] source
cdef double[:] target
cdef double[:,:] weights
cdef long[:,:] effcons
cdef long i,j

cdef double[:] con
cdef long[:] act
effc = []
S = np.random.uniform(0,1,256)
Wmin = 0.25
Wmax = 0.75
weights = AscToAscWeights((256,256),(16,16))
ws = AscToAscWeights((16,16),(16,16))

#N = np.random.normal(0.5, 0.005, weights.shape)
#N = np.minimum(np.maximum(N, 0.0),1.0)
#weights = np.multiply((Wmin + (Wmax - Wmin)*N), weights)

source = S
target = np.empty(256)
cdef pop
pop = int(np.sqrt(len(source)))

for i in range(weights.shape[0]):
    con = np.asarray(weights[i])
    act = np.nonzero(con)[0]
    effc.append(act)
effcons = np.asarray(effc)
k = 16
gain = 1.0
Z = np.reshape(source, (pop, pop))


@cython.boundscheck(False)
@cython.wraparound(False)
def propagate3():
    cdef int i,j
    cdef double[:,:] Z, R, W, EW
    cdef double[:] RC, EWC
    Z = np.reshape(source, (pop, pop))
    R = np.add.reduceat(np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0), np.arange(0, Z.shape[1], k), axis=1)
    RC = np.reshape(R, R.size)
    for i in range(weights.shape[0]):
        W = np.reshape(weights[i], (Z.shape[0],Z.shape[1]))
        EW = np.add.reduceat(np.add.reduceat(W, np.arange(0, W.shape[0], k), axis=0), np.arange(0, W.shape[1], k), axis=1)
        EWC = np.reshape(EW, EW.size)
        EWC = np.array(EWC) / (1.0*k)
        for j in range(EWC.size):
            target[i] += RC[j] * EWC[j] * gain
            
@cython.boundscheck(False)
@cython.wraparound(False)
def propagate4():
    cdef int i,j
    #cdef double[:,:] Z, R
    #cdef double[:] RC
    #Z = np.reshape(source, (pop, pop))
    #R = np.add.reduceat(np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0), np.arange(0, Z.shape[1], k), axis=1)
    #RC = np.reshape(R, R.size)
    for i in range(ws.shape[0]):
        for j in range(ws.shape[1]):
            target[i] += source[j] * ws[i,j] * gain
            
@cython.boundscheck(False)
@cython.wraparound(False)
def propagate1():
    cdef int i,j
    cdef int[:] t
    cdef int tsize
    tsize = weights.shape[0]
    t = range(tsize)
    for i in t:
        for j in effcons[i]:
            target[i] += weights[i,j] * source[j] 
            
@cython.boundscheck(False)
@cython.wraparound(False)
def propagate2():
    cdef int i,j
    for i in range(weights.shape[0]):
        for j in range(weights.shape[1]):
            target[i] += weights[i,j] * source[j]             

        





In [36]:
%timeit propagate1()       

100 loops, best of 3: 12.5 ms per loop


In [37]:
%timeit propagate2()       

10 loops, best of 3: 54.2 ms per loop


In [5]:
%timeit propagate3()       

10 loops, best of 3: 56.9 ms per loop


In [10]:
%timeit propagate4()       

100 loops, best of 3: 15.7 ms per loop
